In [ ]:
import pulp
import collections

# 定义作物和土地类型的全局常量，以增强代码可读性
# Crop Type Codes based on problem description |c|
CROP_TYPE_FUNGUS = 0
CROP_TYPE_GRAIN = 1
CROP_TYPE_VEG_NORMAL = 2
CROP_TYPE_RICE = 3
CROP_TYPE_VEG_SPECIAL = 4

# Land Type Codes based on problem description l
LAND_TYPE_DRY_FLAT = 1
LAND_TYPE_DRY_TERRACE = 2
LAND_TYPE_DRY_HILL = 3
LAND_TYPE_IRRIGATED = 4
LAND_TYPE_GREENHOUSE_NORMAL = 5
LAND_TYPE_GREENHOUSE_SMART = 6

# 设置一个足够大的常数 M，用于“大M法”建模
BIG_M = 1e9


def solveFarmingOptimization(
    num_years,
    num_crops,
    num_lands,
    costs,
    prices,
    yields,
    expected_sales,
    land_sizes,
    crop_types,
    land_types,
    initial_planting_plan,
    min_area_per_crop,
):
    """
    通过构建和求解一个混合整数线性规划模型来解决农业种植优化问题。

    Args:
        num_years (int): T, 计划的总年数。
        num_crops (int): M, 作物的种类数量。
        num_lands (int): N, 地块的数量。
        costs (dict): C_mn, 种植成本。键: (m, n), 值: float。
        prices (dict): p_m, 销售单价。键: m, 值: float。
        yields (dict): Y_mn, 亩产量。键: (m, n), 值: float。
        expected_sales (dict): E_ms, 预期销量。键: (m, s), 值: float。
        land_sizes (dict): b_n, 各地块的面积。键: n, 值: float。
        crop_types (dict): c_m, 各作物的类型代码。键: m, 值: int。
        land_types (dict): l_n, 各地块的类型代码。键: n, 值: int。
        initial_planting_plan (dict): X_0,s,m,n, 第0年的种植计划。键: (s, m, n), 值: float。
        min_area_per_crop (float): epsilon, 每种作物的最小种植面积。

    Returns:
        一个元组，包含:
        - status (str): 求解状态 ('Optimal', 'Infeasible', etc.)。
        - total_profit (float): 最优的总利润。
        - optimal_plan (dict): 最优的种植计划。键: (t, s, m, n), 值: float。
    """

    # --- 步骤 0: 数据预处理与集合定义 ---

    # 定义时间、作物和地块的索引（使用0-based索引）
    YEARS = list(range(1, num_years + 1))
    SEASONS = list(range(1, 3))  # 第1季和第2季
    CROPS = list(range(num_crops))
    LANDS = list(range(num_lands))

    # 根据类型代码创建作物集合
    CROPS_FUNGUS = {m for m, c in crop_types.items() if c == CROP_TYPE_FUNGUS}
    CROPS_GRAIN = {m for m, c in crop_types.items() if abs(c) == CROP_TYPE_GRAIN}
    CROPS_VEG_NORMAL = {
        m for m, c in crop_types.items() if abs(c) == CROP_TYPE_VEG_NORMAL
    }
    CROPS_RICE = {m for m, c in crop_types.items() if c == CROP_TYPE_RICE}
    CROPS_VEG_SPECIAL = {m for m, c in crop_types.items() if c == CROP_TYPE_VEG_SPECIAL}
    CROPS_BEAN = {m for m, c in crop_types.items() if c < 0}

    # 根据类型代码创建地块集合
    LANDS_DRY = {
        n
        for n, l in land_types.items()
        if l in [LAND_TYPE_DRY_FLAT, LAND_TYPE_DRY_TERRACE, LAND_TYPE_DRY_HILL]
    }
    LANDS_IRRIGATED = {n for n, l in land_types.items() if l == LAND_TYPE_IRRIGATED}
    LANDS_GREENHOUSE_NORMAL = {
        n for n, l in land_types.items() if l == LAND_TYPE_GREENHOUSE_NORMAL
    }
    LANDS_GREENHOUSE_SMART = {
        n for n, l in land_types.items() if l == LAND_TYPE_GREENHOUSE_SMART
    }

    # 根据第0年的计划，计算初始的种植状态 (Z_0)
    initial_is_planted = collections.defaultdict(int)
    for (s, m, n), area in initial_planting_plan.items():
        if area > 1e-6:  # 使用一个小的容忍度
            initial_is_planted[s, m, n] = 1

    # --- 步骤 1: 初始化模型 ---
    model = pulp.LpProblem("Farming_Optimization_Model", pulp.LpMaximize)

    # --- 步骤 2: 定义决策变量 ---

    # 连续变量: X_t,s,m,n (种植面积)
    planting_area = pulp.LpVariable.dicts(
        "PlantingArea", (YEARS, SEASONS, CROPS, LANDS), lowBound=0, cat="Continuous"
    )

    # 二进制变量: Z_t,s,m,n (1表示种植, 0表示不种)
    is_planted = pulp.LpVariable.dicts(
        "IsPlanted", (YEARS, SEASONS, CROPS, LANDS), cat="Binary"
    )

    # 连续变量: S_t,s,m (作物的销售量)
    sales_quantity = pulp.LpVariable.dicts(
        "SalesQuantity", (YEARS, SEASONS, CROPS), lowBound=0, cat="Continuous"
    )

    # 二进制变量: W_t,n (水浇地的种植选择: 1为水稻, 0为蔬菜)
    is_irrigated_rice_option = pulp.LpVariable.dicts(
        "IsIrrigatedRiceOption", (YEARS, LANDS_IRRIGATED), cat="Binary"
    )

    # --- 步骤 3: 定义目标函数 ---
    total_revenue = pulp.lpSum(
        prices[m] * sales_quantity[t][s][m]
        for t in YEARS
        for s in SEASONS
        for m in CROPS
    )
    total_cost = pulp.lpSum(
        costs.get((m, n), 0) * planting_area[t][s][m][n]
        for t in YEARS
        for s in SEASONS
        for m in CROPS
        for n in LANDS
    )
    model += total_revenue - total_cost, "Total_Profit"

    # 目标函数线性化约束: S <= min(E, Y*X)
    for t in YEARS:
        for s in SEASONS:
            for m in CROPS:
                model += sales_quantity[t][s][m] <= expected_sales.get((m, s), 0)
                model += sales_quantity[t][s][m] <= pulp.lpSum(
                    yields.get((m, n), 0) * planting_area[t][s][m][n] for n in LANDS
                )

    # --- 步骤 4: 添加所有约束 ---

    # 约束1: 最小种植面积 (关联 X 和 Z)
    for t in YEARS:
        for s in SEASONS:
            for m in CROPS:
                for n in LANDS:
                    model += (
                        planting_area[t][s][m][n]
                        >= min_area_per_crop * is_planted[t][s][m][n]
                    )
                    model += (
                        planting_area[t][s][m][n]
                        <= land_sizes[n] * is_planted[t][s][m][n]
                    )

    # 约束2: 地块面积上限
    for t in YEARS:
        for s in SEASONS:
            for n in LANDS:
                model += (
                    pulp.lpSum(planting_area[t][s][m][n] for m in CROPS)
                    <= land_sizes[n]
                )

    # 约束3: 豆类轮作
    for n in LANDS:
        # 覆盖 [0, 1, 2] 的窗口
        if num_years >= 2:
            beans_y0 = pulp.lpSum(
                initial_is_planted[s, m, n] for s in SEASONS for m in CROPS_BEAN
            )
            beans_y12 = pulp.lpSum(
                is_planted[t][s][m][n]
                for t in [1, 2]
                if t in YEARS
                for s in SEASONS
                for m in CROPS_BEAN
            )
            model += beans_y0 + beans_y12 >= 1

        # 覆盖 [t', t'+1, t'+2] 的窗口 (t' >= 1)
        for t_prime in range(1, num_years - 1):
            if t_prime + 2 in YEARS:
                model += (
                    pulp.lpSum(
                        is_planted[t][s][m][n]
                        for t in range(t_prime, t_prime + 3)
                        for s in SEASONS
                        for m in CROPS_BEAN
                    )
                    >= 1
                )

    # 约束4: 旱地种植
    for t in YEARS:
        for n in LANDS_DRY:
            model += pulp.lpSum(is_planted[t][2][m][n] for m in CROPS) == 0
            non_grain_crops = [m for m in CROPS if m not in CROPS_GRAIN]
            model += pulp.lpSum(is_planted[t][1][m][n] for m in non_grain_crops) == 0

    # 约束5: 水浇地种植 (大M法)
    for t in YEARS:
        for n in LANDS_IRRIGATED:
            w = is_irrigated_rice_option[t][n]  # 1=水稻, 0=蔬菜

            # 水稻选项 (w=1时激活)
            non_rice_crops = [m for m in CROPS if m not in CROPS_RICE]
            model += pulp.lpSum(
                is_planted[t][s][m][n] for s in SEASONS for m in non_rice_crops
            ) <= len(non_rice_crops) * 2 * (1 - w)
            for m in CROPS_RICE:
                model += is_planted[t][1][m][n] + is_planted[t][2][m][n] <= 1

            # 蔬菜选项 (w=0时激活)
            model += (
                pulp.lpSum(is_planted[t][s][m][n] for s in SEASONS for m in CROPS_RICE)
                <= len(CROPS_RICE) * 2 * w
            )
            non_normal_veg = [m for m in CROPS if m not in CROPS_VEG_NORMAL]
            model += (
                pulp.lpSum(is_planted[t][1][m][n] for m in non_normal_veg)
                <= len(non_normal_veg) * w
            )
            non_special_veg = [m for m in CROPS if m not in CROPS_VEG_SPECIAL]
            model += (
                pulp.lpSum(is_planted[t][2][m][n] for m in non_special_veg)
                <= len(non_special_veg) * w
            )
            model += pulp.lpSum(is_planted[t][2][m][n] for m in CROPS_VEG_SPECIAL) <= 1

    # 约束6: 特殊蔬菜种植
    for t in YEARS:
        for m in CROPS_VEG_SPECIAL:
            non_irrigated_lands = [n for n in LANDS if n not in LANDS_IRRIGATED]
            model += (
                pulp.lpSum(
                    is_planted[t][s][m][n] for s in SEASONS for n in non_irrigated_lands
                )
                == 0
            )
            model += pulp.lpSum(is_planted[t][1][m][n] for n in LANDS) == 0

    # 约束7: 普通大棚种植
    for t in YEARS:
        for n in LANDS_GREENHOUSE_NORMAL:
            non_normal_veg = [m for m in CROPS if m not in CROPS_VEG_NORMAL]
            model += pulp.lpSum(is_planted[t][1][m][n] for m in non_normal_veg) == 0
            non_fungus = [m for m in CROPS if m not in CROPS_FUNGUS]
            model += pulp.lpSum(is_planted[t][2][m][n] for m in non_fungus) == 0

    # 约束8: 食用菌种植
    for t in YEARS:
        for m in CROPS_FUNGUS:
            non_normal_gh = [n for n in LANDS if n not in LANDS_GREENHOUSE_NORMAL]
            model += (
                pulp.lpSum(
                    is_planted[t][s][m][n] for s in SEASONS for n in non_normal_gh
                )
                == 0
            )
            model += pulp.lpSum(is_planted[t][1][m][n] for n in LANDS) == 0

    # 约束9: 智慧大棚种植
    for t in YEARS:
        for n in LANDS_GREENHOUSE_SMART:
            non_normal_veg = [m for m in CROPS if m not in CROPS_VEG_NORMAL]
            model += (
                pulp.lpSum(
                    is_planted[t][s][m][n] for s in SEASONS for m in non_normal_veg
                )
                == 0
            )

    # 约束10: 禁止重茬
    for m in CROPS:
        for n in LANDS:
            # 年内重茬
            for t in YEARS:
                model += is_planted[t][1][m][n] + is_planted[t][2][m][n] <= 1
            # 跨年重茬 (S2 -> S1)
            model += initial_is_planted[2, m, n] + is_planted[1][1][m][n] <= 1
            for t in range(1, num_years):
                model += is_planted[t][2][m][n] + is_planted[t + 1][1][m][n] <= 1
            # 旱地跨年重茬 (S1 -> S1)
            if n in LANDS_DRY:
                model += initial_is_planted[1, m, n] + is_planted[1][1][m][n] <= 1
                for t in range(1, num_years):
                    model += is_planted[t][1][m][n] + is_planted[t + 1][1][m][n] <= 1

    # --- 步骤 5: 求解模型 ---
    # PuLP会使用其默认的CBC求解器，无需额外安装。
    # model.writeLP("FarmingModel.lp") # 可选：将模型写入文件进行调试
    model.solve()

    # --- 步骤 6: 返回结果 ---
    status = pulp.LpStatus[model.status]
    total_profit = pulp.value(model.objective) if status == "Optimal" else 0.0

    optimal_plan = collections.defaultdict(float)
    if status == "Optimal":
        for t in YEARS:
            for s in SEASONS:
                for m in CROPS:
                    for n in LANDS:
                        area = planting_area[t][s][m][n].varValue
                        if area is not None and area > 1e-6:
                            optimal_plan[(t, s, m, n)] = area

    return status, total_profit, optimal_plan


def createSampleData():
    """创建一个小规模、逻辑一致的样本数据集用于测试。(修正版)"""
    T, M, N = 3, 6, 4  # <--- 作物数量增加到6
    CROP_IDS = list(range(M))
    LAND_IDS = list(range(N))

    # 作物类型: 0:食用菌, 1:小麦(粮), 2:玉米(豆/粮), 3:白菜(普菜), 4:水稻, 5:豌豆(豆/普菜)
    CROP_TYPES = {0: 0, 1: 1, 2: -1, 3: 2, 4: 3, 5: -2}  # <--- 新增作物5，类型为-2

    # 土地类型: 0:旱地, 1:水浇地, 2:普通大棚, 3:智慧大棚
    LAND_TYPES = {0: 3, 1: 4, 2: 5, 3: 6}

    costs = collections.defaultdict(float)
    prices = {0: 15.0, 1: 2.5, 2: 3.0, 3: 1.5, 4: 4.0, 5: 5.0}  # <--- 新增作物5的价格
    yields = collections.defaultdict(float)
    expected_sales = collections.defaultdict(float)
    land_sizes = {0: 50.0, 1: 100.0, 2: 20.0, 3: 30.0}

    for m in CROP_IDS:
        for s in [1, 2]:
            expected_sales[m, s] = 50000.0
    for m in CROP_IDS:
        for n in LAND_IDS:
            costs[m, n] = 100 + m * 10 + n * 5
            yields[m, n] = 1000 + m * 50 - n * 10

    # 第0年计划: 在旱地上种了40亩小麦
    initial_plan = collections.defaultdict(float)
    initial_plan[1, 1, 0] = 40.0

    min_area = 1.0

    return (
        T,
        M,
        N,
        costs,
        prices,
        yields,
        expected_sales,
        land_sizes,
        CROP_TYPES,
        LAND_TYPES,
        initial_plan,
        min_area,
    )

In [ ]:
if __name__ == "__main__":
    # 获取修正后的样本数据
    (
        T,
        M,
        N,
        costs,
        prices,
        yields,
        expected_sales,
        land_sizes,
        CROP_TYPES,
        LAND_TYPES,
        initial_plan,
        min_area,
    ) = createSampleData()

    # 调用函数求解
    print("开始求解农业规划优化问题...")
    status, profit, plan = solveFarmingOptimization(
        num_years=T,
        num_crops=M,
        num_lands=N,
        costs=costs,
        prices=prices,
        yields=yields,
        expected_sales=expected_sales,
        land_sizes=land_sizes,
        crop_types=CROP_TYPES,
        land_types=LAND_TYPES,
        initial_planting_plan=initial_plan,
        min_area_per_crop=min_area,
    )

    # 打印和格式化输出结果
    print("-" * 50)
    print(f"求解状态: {status}")
    if status == "Optimal":
        print(f"最优总利润: {profit:,.2f} 元")
        print("\n最优种植方案:")
        if not plan:
            print("计算结果为不进行任何种植。")
        else:
            sorted_plan = sorted(plan.items())
            for (t, s, m, n), area in sorted_plan:
                print(
                    f"  第 {t} 年 - 第 {s} 季: 在地块 {n} 种植作物 {m}, 面积: {area:.2f} 亩"
                )
    else:
        print("未能找到最优解。可能的原因是模型无解或存在其他问题。")
    print("-" * 50)

开始求解农业规划优化问题...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/joyce/.pyenv/versions/3.12.11/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/e25a3b512beb465aa2feb6af6b17f8ee-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e25a3b512beb465aa2feb6af6b17f8ee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 511 COLUMNS
At line 2150 RHS
At line 2657 BOUNDS
At line 2781 ENDATA
Problem MODEL has 506 rows, 273 columns and 1242 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

--------------------------------------------------
求解状态: Infeasible
未能找到最优解。可能的原因是模型无解或存在其他问题。
--------------------------------------------------
